# Package

In [ ]:
#plotly
import chart_studio
import plotly
import plotly.graph_objs as go
import plotly.express as px
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.io as po

#folium
from folium.plugins import MarkerCluster
import folium 
import numpy as np
import json
import branca 
from branca.colormap import linear
import branca.colormap as cmp
import geopandas as gpd


import pandas as pd
pd.set_option('display.max_rows', None)  # 행 펼치기

import numpy as np
import re
import matplotlib.pyplot as plt
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드


# Data load

In [ ]:
df = pd.read_csv("c:\\spc\\spc_geo_final_230222.csv")

In [ ]:

#세종특별자치시는 시구군이 공백으로 채워져 있어 이를 세종특별자치시로 채워줍니다.

df_fill = df.copy()

df_fill.sigugun.fillna('', inplace = True)

pt = re.compile('자치시')

for idx in df_fill.index:
    if len(re.findall(pt, df_fill.loc[idx,'sido'])) > 0:
        if len(df_fill.loc[idx,'sigugun']) == 0:
            df_fill.loc[idx,'sigugun'] = df_fill.loc[idx,'sido']
        else:
            df_fill.loc[idx,'sigugun'] = f"{df_fill.loc[idx,'sido']} {re.sub('NaN','',df_fill.loc[idx,'sigugun'])}"


# df_fill.head(10) # 확인

# Treemap
plotly를 이용하여 트리맵을 생성합니다. 

In [ ]:
df_trm_region =  df_fill.groupby(['sido', 'sigugun','dongmyun','brand'])[['addr']].count().sort_values(by = ['sido','addr'], ascending = [True, False]).reset_index()
df_trm_region.rename(columns = {'addr':'cnt'}, inplace = True)

## 브랜드별 매장수

In [ ]:
fig1 = px.treemap(df_trm_region,
                 path = [
                     px.Constant('All'),
                     'brand','sido','sigugun','dongmyun'],
                 values = 'cnt',
                 maxdepth = 2,
                )

#title
fig1.update_layout(title={'text' :'<b> SPC 브랜드 매장수</b><br><sup>브랜드별</sup>',
                         'y' : 0.93,
                         'x' : 0.5,
                        'xanchor' : 'center',
                        'yanchor' : 'top'},
                  font = dict(family="Calibri",
                              size = 13,
                              color='#000000'),
                  showlegend = False,
                  margin = dict(l=20,r=20,t=80,b=80),
                  title_font_color = 'white',
                  template="plotly_white",
                  paper_bgcolor="#444444",
                 )

# add percent info
percents = [val/sum(df_trm_region.cnt) * 100 for val in fig1.data[0].values]
fig1.data[0].customdata = np.column_stack([percents])

# hover
fig1.layout.coloraxis.colorbar.title = '매장수(개)'
fig1.data[0].hovertemplate = """
<b>Path :</b> %{currentPath}<br>
<b>Label :</b> %{label}<br>
<b>Cnt :</b> %{value}개<br>
<b>Pct :</b> %{customdata[0]:.2f}%"""
# cf) pct에 원래는 percentRoot 사용하면 되는데 이상하게 1%가 전체로 잡혀서 백분율이 제대로 표현되지 않아 퍼센트 값을 따로 입력해주었음


# inside text annotation
fig1.update_traces(textinfo = "label",
                  textposition="middle center",
                  texttemplate='<b>%{label: labels}</b><br><b>%{value: values}개</b>',
                  insidetextfont = {'size': 15},              
                )

# annotation
fig1.add_annotation(
  text=("""
* 데이터 기준 : 2023년 2월 22일<br>
* Box 위에 마우스 커서를 올리면 정보를 확인할 수 있습니다.<br>
* Box를 클릭하면 시도별,시구군별 매장수를 확인할 수 있습니다. """
       ),
  xref="paper", yref="paper",
  x=0.01, y=-0.13, showarrow=False,
  font=dict(
      family="Calibri",
      size=11,
      color='white'
  ),
    opacity=0.8,
    align="left",
)

fig1.show()

# save 
# po.write_html(fig, file='input path', auto_open=False)

## 지역별 매장수

In [ ]:
fig2 = px.treemap(df_trm_region,
                  path = [
                    px.Constant('All'),
                    'sido','sigugun','dongmyun','brand'],
                  values = 'cnt',
                 )

# background color
fig2.update_traces(root_color="lightgrey")  

#title
fig2.update_layout(title={'text' :'<b> SPC 브랜드 매장수</b><br><sup>지역별</sup>',
                          'y' : 0.96,
                          'x' : 0.5,
                          'xanchor' : 'center',
                          'yanchor' : 'top'},
                   font = dict(family="Arial",
                               size = 13,
                               color='#000000'),
                   showlegend = False,
                   margin = dict(l=10,r=10,b=80),
                  )

# add percent info
percents = [val/sum(df_trm_region.cnt) * 100 for val in fig2.data[0].values]
fig2.data[0].customdata = np.column_stack([percents])

fig2.layout.coloraxis.colorbar.title = '매장수(개)'
fig2.data[0].hovertemplate = '<b>%{label}</b><br>%{value}개(%{customdata[0]:.2f}%)'


# annotation
fig2.add_annotation(
    text=("""
* 데이터 기준 : 2023년 2월 22일<br>
* Box 위에 마우스 커서를 올리면 정보를 확인할 수 있습니다.<br>
* Box를 클릭하면 브랜별 매장수를 확인할 수 있습니다. """
       ),
    xref="paper", yref="paper",
    x=0.01, y=-0.13, showarrow=False,
    font=dict(
        family="Calibri",
        size=11,
        color='black'
    ),
    opacity=0.8,
    align="left",
)

fig2.show()

# save
# po.write_html(fig, file='input path', auto_open=False)

# Choropleth
folium을 이용하여 choropleth를 생성합니다.<br>
이 작업을 수행하기 전에 시군구별 geojson 파일을 다운받아야 합니다.<br>
[링크](https://mjs1995.tistory.com/169?category=802136)를 누르시면 geojson 다운 받는 방법이 자세히 나와있습니다.

In [ ]:
#특별시, 광역시, 자치시의 경우 시구군에 중복되는 명칭이 중복되어 이를 방지하기 위해서 시도명을 앞에 붙여줘야함

df_map = df.copy()

import re
df_map.sigugun.fillna('', inplace = True)

pt = re.compile('특별시|광역시|자치시')

for idx in df_map.index:
    if len(re.findall(pt, df_map.loc[idx,'sido'])) > 0:
        if len(df_map.loc[idx,'sigugun']) == 0:
            df_map.loc[idx,'sigugun'] = df_map.loc[idx,'sido']
        else:
            df_map.loc[idx,'sigugun'] = f"{df_map.loc[idx,'sido']} {re.sub('NaN','',df_map.loc[idx,'sigugun'])}"


df_map.head(10)

## 전체 매장수
이때 사용된 데이터셋은 pandas로 나중에 브랜드별로 사용되는 choropleth의 지도의 geopandas와 데이터셋이 다르다.

|datasets|columns |
|---|:---|
|code_sgg|- SIG_CD : 코드 넘버
||- SIG_ENG_NM : 시군구 영문명
||- SIG_KOR_NM : 시군구 한글명|
|geo_nation|- sigugun : 시군구 한글명|
||- cnt : 시군구별 총 매장수|

In [ ]:
# geojson파일 로드 <- 파일 json으로 로드함
state_path = 'c:\\spc\\map_sigugun.geojson'
state_name = '시구군'

with open(state_path, encoding = 'utf-8') as f:
    state_geo = json.load(f)

In [ ]:
#시군구 코드가 담긴 csv 파일
code_sgg = pd.read_csv("c:\\spc\\map_sigugun_newcode.csv", encoding = 'euc-kr') 

In [ ]:
# 필요 데이터 추출
geo_nation = df_map.groupby(['sigugun'])[['addr']].count().reset_index()
geo_nation.rename(columns = {'addr':'cnt'}, inplace = True)
geo_nation.head(10)

In [ ]:
# 전국 매장수 브랜드 total

center = [36.9523500,127.0769624]
m = folium.Map(
    location=center,
    zoom_start=10,
    min_zoom = 6,
    tiles='cartodbpositron'
)

folium.GeoJson(state_geo,name=state_name,).add_to(m)

choropleth = folium.Choropleth(geo_data=state_geo,
                               data=geo_nation,
                               columns=["sigugun", "cnt"],
                               fill_color='YlOrRd', # 색상 변경
                               threshold_scale=[0, 10, 30, 50, 80, 100,120],
                               nan_fill_color='gray',
                               fill_opacity=0.8, # 색 투명도
                               line_weight = 1, # 라인 두께
                               key_on='properties.SIG_KOR_NM',
                               legend_name="시도별 SPC 매장수 (단위 : 개)",
                              ).add_to(m)

# SIG_KOR_NM 수정
for ft in choropleth.geojson.data['features']:
    addr = code_sgg[code_sgg['SIG_CD'] == int(ft['properties']['SIG_CD'])]['SIG_KOR_NM'].values[0]
    ft['properties']['SIG_KOR_NM'] = addr


# CNT 정보 추가
for ft in choropleth.geojson.data['features']:
    try:
        ft['properties']['cnt'] = int(geo_nation['cnt'][geo_nation.sigugun== ft['properties']['SIG_KOR_NM']].values[0])
    except:
        ft['properties']['cnt'] = 0
    

choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(
            fields=['SIG_KOR_NM','cnt'],
            aliases=['SIGUGUN','STORE_CNT'],
            localize=True,
        )
)


m

# save
# m.save('input path')


## 브랜드별 매장수 
이때 동일한 geojson파일을 geopandas로 로드하여 데이터 처리합니다.

|dataset|columns|
|---|:---|
|geo_br|- sido : 시도 한글명|
||- sigugun : 시구군 한글명|
||- brand : 브랜드명|
||- cnt : 매장수|
|geojson|- SIG_CD : 코드 넘버|
||- SIG_KOR_NM : 시구군 한글명|
||- geometry : 지역 경계선 좌표|
|geo_final|-SIG_CD : 코드 넘버|
||- SIG_KOR_NM : 시구군 한글명|
||- geometry : 지역 경계선 좌표|
||- sido : 시도 한글명|
||- sigugun : 시구군 한글명|
||- brand : 브랜드명|
||- cnt : 매장수|

In [ ]:
# 필요한 데이터 추출
geo_br = df_map.groupby(['sido','sigugun','brand'])[['addr']].count().reset_index()
geo_br.rename(columns = {'addr':'cnt'}, inplace = True)
geo_br.head(10)

In [ ]:
# geopandas를 이용해 geojson파일 로드
geojson = gpd.read_file('c:\\spc\\map_sigugun.geojson')

# geojson의 SIG_KOR_NM 수정
for idx, code in zip(geojson.index, geojson.SIG_CD):
    code_nm = code_sgg[code_sgg['SIG_CD'] == int(code)]['SIG_KOR_NM'].values[0]
    geojson.loc[idx, 'SIG_KOR_NM'] = code_nm

# 필요한 컬럼 선별
geojson = geojson[['SIG_CD','SIG_KOR_NM','geometry']]
geojson.head(10)

In [ ]:
# geojson파일과 데이터셋(df_map2) 병합
# 반드시 geojson파일로 병합해야함
geo_final = geojson.merge(geo_br,left_on = 'SIG_KOR_NM', right_on = 'sigugun', how = 'outer')
geo_final = geo_final[~geo_final['geometry'].isna()]
geo_final.head()

In [ ]:
# null값 확인
df_null = geo_final[geo_final.brand.isnull()]  #청송군, 울릉군 null값 확인
for idx in df_null.index:
    geo_final.loc[idx, 'sigugun'] = geo_final.loc[idx,'SIG_KOR_NM']
    geo_final.loc[idx,'brand'] = ''
    geo_final.loc[idx,'cnt'] = 0

geo_final[geo_final.cnt.isnull()] #check

In [ ]:
def MakeChoropleth(geodata, commodities, fs):
    choropleth = folium.Choropleth(
        geo_data = geodata,
        name='choropleth',
        data=commodities,
        columns=['sigugun','cnt'],
        key_on='properties.SIG_KOR_NM',
        fill_color='YlOrRd',
        threshold_scale= colormap.index,
        nan_fill_color="gray",
        fill_opacity=0.7,
        line_weight =1,
        legend_name='시구군별 SPC 매장수(단위 : 개)',
        highlight=True,
        ).geojson.add_to(fs)
    
    
    return choropleth

def LabelCnt(choropleth,commodity):
    folium.features.GeoJson(
                    data=commodity,
                    name='brand',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['brand',
                                'sigugun',
                                'cnt',
                               ],
                        aliases=['BRAND : ',
                                 'REGION : ',
                                 'STORE_CNT :',
                                ],
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth)     

In [ ]:
# 브랜드별 choropleth

center = [36.9523500,127.0769624]

f = folium.Figure(width=1000, height=600)
m = folium.Map(location= center, tiles=None, overlay = False,
               zoom_start=10, min_zoom = 8).add_to(f)

# color
colormap = linear.YlOrRd_09.scale(geo_final.cnt.min(), geo_final.cnt.max()).to_step(6)
colormap.caption = '시도별 SPC 매장수(단위 : 개)'
colormap.add_to(m)

# 변수 생성
BrandList = geo_final.brand.unique().tolist()[:-1]
cnt = len(BrandList)
commodities = []
fs = []

for i,br in enumerate(BrandList):
    globals()[f'commodity_{i}'] = geo_final[geo_final.brand == br]
    commodities.append(globals()[f'commodity_{i}'])
    
    globals()[f'feature_group_{i}'] = folium.FeatureGroup(name = br, overlay=False).add_to(m)
    fs.append(globals()[f'feature_group_{i}'])


# choropleth 만들기
choropleths = []
for i in range(cnt):
    globals()[f'choropleth_{i}'] = MakeChoropleth(state_geo,commodities[i], fs[i])
    choropleths.append(globals()[f'choropleth_{i}'])

#     # SIG_KOR_NM 수정
#     for ft in globals()[f'choropleth_{i}'].data['features']:
#         addr = code_sgg[code_sgg['SIG_CD'] == int(ft['properties']['SIG_CD'])]['SIG_KOR_NM'].values[0]
#         ft['properties']['SIG_KOR_NM'] = addr
        

# 라벨링
for i in range(cnt):
    LabelCnt(choropleths[i], commodities[i])
    
# mode 설정
folium.TileLayer('cartodbdark_matter',overlay=True,name="dark mode").add_to(m)
folium.TileLayer('cartodbpositron',overlay=True,name="light mode").add_to(m)
folium.LayerControl(collapsed=False).add_to(m)


m

# save
# m.save('c:\\spc\\final\\spc_choropleth_brand.html')

# Bar graph
plotly, matplotlib를 이용하여 Bar graph를 생성합니다.

In [ ]:
#bar graph를 그리기 위한 값 보정

bar_pre = geo_final.copy()

bar_pre['sido'][bar_pre.SIG_KOR_NM == '청송군'] ='경상북도'
bar_pre['sido'][bar_pre.SIG_KOR_NM == '울릉군'] ='경상북도'
bar_pre['brand'][bar_pre.SIG_KOR_NM == '청송군'] = '파리바게뜨'
bar_pre['brand'][bar_pre.SIG_KOR_NM == '울릉군'] = '파리바게뜨'

bar_pre[bar_pre.sido.isnull()]

In [ ]:
# dataset

bar_final = bar_pre.groupby(['sido','sigugun','brand'])[['cnt']].sum().reset_index()
bar_final.cnt = bar_final.cnt.astype(int)
bar_final.head(10)

## 시구군별/브랜드별 매장수 그래프
plotly를 이용하여 막대그래프를 생성하였다. 버튼기능을 넣어 동적으로 변환 가능

In [ ]:
def draw_graph(label, x, y, color):
    
    fig = go.Bar(
                name = label,
                x = x,
                y = y, 
                marker = dict(color = color),
                visible = False,
                orientation='h',        
                )
    
    return fig


def write_sumcnt(x,y):
    fig = go.Scatter(
        x = x, 
        y = y,
        text = x,
        name = 'total',
        mode = 'text',
        textposition = 'middle right',
        textfont = dict(
            size = 12,
        ),
        showlegend = False,
        visible = False,
    )
    return fig


def btn_dropdown(label, visible_list):
    btn_dict = dict(label = label,
                    method = "update",
                    args = [{"visible" : visible_list },
                            {"title" : f'SPC 매장수 : {label}'}
                           ])
    
    return btn_dict    

In [ ]:
# 변수생성
cnt = 0
sumnum = (len(bar_final.brand.unique()) * len(bar_final.sido.unique())) + len(bar_final.sido.unique())
visible = [False]*sumnum
pt = list(zip(*[iter([i for i in range(sumnum)])]*(len(bar_final.brand.unique())+1)))  # brand 갯수 + total 텍스트 그래프

# 컬러개수 22개
# 브랜드 대표 컬러로 지정
colors = ['orange', 'hotpink', 'black', 'cornflowerblue', 'crimson', 'darkgoldenrod', 'forestgreen', 'yellowgreen',
       'sienna', 'darkviolet', 'bisque', 'teal', 'navy', 'purple', 'yellow', 'tan',
       'olive', 'silver', 'steelblue', 'aqua', 'magenta','goldenrod']

data = []
button = [ dict(label = '선택',
                method = "update",
                args = [{"visible" : visible},
                            {"title" : ''}])]

# 그래프 생성 + 버튼 설정
for pi, (con, group) in enumerate(bar_final.groupby('sido')): 
    for i, (br, p) in enumerate(zip(bar_final.brand.unique(), pt[pi])):
        bar = draw_graph(br, group['cnt'][group.brand == br], group['sigugun'][group.brand == br], colors[i])
        data.append(bar)
        
        visible[p] = True
        
    w = group.groupby('sigugun')[['cnt']].sum()
    wscatter = write_sumcnt(w.cnt, w.index)
    data.append(wscatter)
    visible[pt[pi][-1]] = True
    
    btn = btn_dropdown(con, visible)
    button.append(btn)
    visible = [False]*sumnum
        
        
# 버튼 위치
updatemenus = [
    dict(
    buttons = list(button),
    direction = "down",
        x= 0.98,
        y=1.08
    ),
]
        
# 레이아웃 설정
layout = dict(title='<b>SPC 지역별 매장수 그래프</b><br><sup>지역을 선택하세요.</sup>', showlegend=True,
              updatemenus = updatemenus, barmode = "stack",
              font = dict(family="Calibri",) 
             )

# 그래프 그리기
fig = go.Figure(data=data, layout=layout)
fig.update_layout(height = 900, width = 800)


fig.show()

# save
# plotly.offline.plot(fig, filename='input path', auto_open = False)

## 시도별 매장수
matplotlib를 이용하여 그래프를 생성합니다.<br>
이때 데이터셋은 트리맵에서 사용했던 df_trm_region에서 데이터를 추출해서 생성합니다.

In [ ]:
bar_reg = df_trm_region.groupby('sido')[['cnt']].sum().reset_index()
bar_reg.sort_values(by = 'cnt', ascending = False, inplace = True)

In [ ]:
# 전체 매장수
plt.figure(figsize = (20, 10))
ax = plt.bar(bar_reg.sido, bar_reg.cnt ,width = 0.6,align='center', color = 'lavender')
ax[0].set_color('darkblue')

# 평균 매장수
ax2 = plt.bar(bar_reg.sido, bar_reg.cnt ,width = 0.6,align='center', color = 'lavender')
ax2[0].set_color('darkblue')

# 전국 평균 라인
plt.axhline(bar_reg.cnt.mean(), color='darkred', linestyle='--', linewidth=1)
plt.text(int(bar_reg.sido.count())/2,bar_reg.cnt.mean() +20,f'전국 시도별 평균 {int(bar_reg.cnt.mean())}개', fontsize = 13,color='darkred' )



plt.bar_label(ax, size = 13)
plt.title('시도별 SPC 매장수', size = 25, pad = 20)
# plt.grid(True, axis = 'y',  alpha=0.2)
plt.xticks(size = 13, rotation = 30)
plt.yticks(size = 13)
plt.ylabel('매장수(단위 : 개)',size = 12)

# font = {'family': 'Arial',
#       'color':  'black',
#         'size': 15,
#       'alpha': 0.7}

for i in list(['right','top']):
    plt.gca().spines[i].set_visible(False)

plt.show()

## 시구군별 매장수

In [ ]:
bar_sgg = df_trm_region.groupby(['sido','sigugun'])[['cnt']].sum().reset_index()
bar_sgg.sort_values(by = 'cnt', ascending = False, inplace = True)

# 상위 10개만 따로
bar_sgg_top10 = bar_sgg.head(10)
bar_sgg_top10['addr'] = bar_sgg_top10['sido']+' '+bar_sgg_top10['sigugun']

# 수도권만 따로
bar_sgg_sudo = bar_sgg[bar_sgg.sido.apply(lambda x: x in ['인천광역시', '서울특별시', '경기도'])]

In [ ]:
# 그래프 그리기
plt.figure(figsize = (20, 10))
ax = plt.bar(bar_sgg_top10.addr, bar_sgg_top10.cnt ,width = 0.6,align='center', color = 'cornsilk')
ax[0].set_color('goldenrod')

# 전국 평균
plt.axhline(bar_sgg.cnt.mean(), color='darkred', linestyle='--', linewidth=1)
plt.text(int(bar_sgg_top10.addr.count())/2,bar_sgg.cnt.mean() +2,f'전국 시구군별 평균 {int(bar_sgg.cnt.mean())}개', fontsize = 13,color='darkred' )

# 수도권 평균
plt.axhline(bar_sgg_sudo.cnt.mean(), color='black', linestyle='--', linewidth=1)
plt.text(int(bar_sgg_top10.addr.count())/3, bar_sgg_sudo.cnt.mean() +1,f'수도권 평균 {int(bar_sgg_sudo.cnt.mean())}개', fontsize = 13,color='black' )

# 축
plt.bar_label(ax, size = 13)
plt.title('시구군별 SPC 매장 TOP10', size = 25, pad = 20)
plt.xticks(size = 13, rotation = 30)
plt.yticks(size = 13)
plt.ylabel('매장수(단위 : 개)',size = 12)

for i in list(['right','top']):
    plt.gca().spines[i].set_visible(False)

plt.show()

## 동면별 매장수

In [ ]:
bar_dong = df_trm_region.groupby(['sido','sigugun','dongmyun'])[['cnt']].sum().reset_index()
bar_dong.sort_values(by = 'cnt', ascending = False, inplace = True)

# 상위 10개만 따로
bar_dong_top10 = bar_dong.head(10)
bar_dong_top10['addr'] = bar_dong_top10['sido']+' '+bar_dong_top10['sigugun']+' '+bar_dong_top10['dongmyun']

# # 수도권만 따로
bar_dong_sudo = bar_dong[bar_dong.sido.apply(lambda x: x in ['인천광역시', '서울특별시', '경기도'])]

In [ ]:
# 그래프 그리기
plt.figure(figsize = (20, 10))
ax = plt.bar(bar_dong_top10.addr, bar_dong_top10.cnt ,width = 0.6,align='center', color = 'beige')
ax[0].set_color('darkgreen')

# 전국 평균
plt.axhline(bar_dong.cnt.mean(), color='darkred', linestyle='--', linewidth=1)
plt.text(int(bar_dong_top10.addr.count())/2, bar_dong.cnt.mean() -2,f'전국 평균 {int(bar_dong.cnt.mean())}개', fontsize = 13,color='darkred' )

# 수도권 평균
plt.axhline(bar_dong_sudo.cnt.mean(), color='black', linestyle='--', linewidth=1)
plt.text(int(bar_dong_top10.addr.count())/3, bar_dong_sudo.cnt.mean() +1,f'수도권 평균 {int(bar_dong_sudo.cnt.mean())}개', fontsize = 13,color='black' )

# 축
plt.bar_label(ax, size = 13)
plt.title('동면별 SPC 매장 TOP10', size = 25, pad = 20)
plt.xticks(size = 13, rotation = 30)
plt.yticks(size = 13)
plt.ylabel('매장수(단위 : 개)',size = 12)

for i in list(['right','top']):
    plt.gca().spines[i].set_visible(False)

plt.show()